In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import itertools as it
import functools as ft
import pyramda as r
import numpy as np
import operator as op
from dataclasses import dataclass
from typing import Any

In [ ]:
import bokeh.plotting as bkp
import bokeh.io as bio
import bokeh.layouts as bkl
import bokeh.models as bkm
bio.output_notebook()

In [458]:
@dataclass
class Composer:
    fn: Any
#     def __post_init__(self):
#         self.fn = r.curry(self.fn)
    @classmethod
    def of(cls,fn):
        return Composer(lambda x: fn)
    # m.chain(f).chain(g) == m.chain(x => f(x).chain(g))
    def chain(self,fn):
        return fn(self.fn)
    def map(self,fn):
        return Composer(fn(self.fn))
    def ap(self,efn):
        return Composer.of(efn.fn(self.fn)(self.fn(self.fn)))

#print( Composer(add(1)).chain(sub(9)).fn(8) )
#print( Composer(add(1)).chain(lambda x: sub(9)).fn(8) )


# fn()
@r.curry      
def add(x,y):
    print(f'adding!')
    return x+y
@r.curry      
def sub(x,y):
    print(f'subbing!')
    return x-y
@r.curry      
def mul(x,y):
    print(f'mulling!')
    return x*y
def dec(fn):
    def wrapper(*args,**kwargs):
        print(f'args {args},{kwargs}')
        result = fn(*args,**kwargs)
        print(f'{result}')
        return result
    return wrapper

def curry(fn):
    @ft.wraps(fn)
    def wrapper(*arg,**kwargs):
        try:
            return fn(*arg,**kwargs)
        except TypeError as e:
            #print(e)
            if hasattr(fn,'func'):
                return curry(ft.partial(fn.func,*fn.args,*arg,**fn.keywords,**kwargs))
            else:
                return curry(ft.partial(fn,*arg,**kwargs))
    return wrapper

class Chainable:
    def __getattribute__(self,name):
        fn = globals()[name]
        def wrapper(*args,**kwargs):
            def compose(ofn):
                return Composer(lambda x: fn(ofn(x)))
            return compose
        return lambda ofn: Composer(lambda x: fn(ofn(x))) if callable(fn) else fn

In [343]:
#C = Chainable()
of = Composer.of
print(of(-199).ap(of(sub)).fn(1)(-1))
print(of(sub(-199)).fn(1)(-1))

# b = of(add(9))
# a = of(sub(3))
# b.ap(a.map(add))

-198
-198


In [185]:
of(lambda x: add).ap(of(lambda x: x(-4) )).fn(1)(9)(8)

17

In [160]:
m = Composer(add(1))
f = lambda val: Composer(lambda x: add(2)(val(x)))
g = lambda val: Composer(lambda x: sub(9)(val(x)))
print( m.chain(f).chain(g).fn(10) )
print( m.chain(lambda x: f(x).chain(g)).fn(10) )
9-(2+(1+10))

-4
-4


-4

In [161]:
m.chain(f).map(dec).chain(g).fn(10)

args (10,),{}
13


-4

In [148]:
# A.of(x).ap(A.of(f)) === A.of(f(x))
out = ( 
Composer(add(9))
    .map(dec)
    .chain(sub(90))
    .map(dec)
#    .ap( of(1).ap( of(sub) ) )
    .map(dec) 
#    .chain(of(90)) 

#    .fn(1)
)
out.fn(10)

args (10,),{}
args (10,),{}
args (10,),{}
19
71
71


71

In [71]:
Composer(lambda x: x).ap(Composer(1) )

Composer(fn=1)

In [375]:
def compose(f,g):
    def wrapper(*args,**kwargs):
        return f(g(*args,**kwargs))
    return wrapper

@dataclass
class IO:
    unsafe_io: Any
    @classmethod
    def of(cls,x):
        return cls(lambda *args,**kwargs: x)
    def map(self,fn):
        return IO(compose(fn,self.unsafe_io))
    def ap(self,f):
        return self.chain(lambda fn: f.map(fn))
    def chain(self,fn):
        return self.map(fn).join()
    def join(self):
        return self.unsafe_io()
    

In [376]:
of = IO.of
io_op0 = of(add).ap(of(1)).ap(of(1))#.ap(of(sub) )#.ap(of(1))#.ap(of(19))
io_op = of(add)
io_op0

AttributeError: 'function' object has no attribute 'ap'

In [374]:
io_op0.unsafe_io(9)

adding!


2

In [371]:
IO.of(add(1)).map(sub(10)).ap(of(8)).unsafe_io(of(9))

subbing!


TypeError: unsupported operand type(s) for -: 'int' and 'function'

In [353]:
io_op0.join()

adding!


20

In [292]:
def encase(fn):
    def wrapper(*args,**kwargs):
        @ft.wraps(fn)
        def get_result():
            try:
                return fn(*args,**kwargs)
            except Exception as e:
                return e
        def fork(reject,resolve):
            result = get_result()
            print(f'result: {result}')
            return resolve(result) if not isinstance(result,Exception) else reject(result)
        get_result.fork = fork
        return get_result
    return wrapper

@r.curry
def fork(io_op,reject,resolve):
    result = io_op.join()
    print(f'result: {result}')
    return resolve(result) if not isinstance(result,Exception) else reject(result)

@dataclass
class Task:
    fork: Any
    @classmethod
    def of(cls,x):
        return cls( lambda _,resolve: resolve(x) )
    def map(self,fn):
        return Task(lambda reject,resolve: self.fork(reject,compose(resolve,fn)))
    def ap(self,f):
        return self.chain(lambda fn: f.map(fn))
    def chain(self,fn):
        return Task(
            lambda reject,resolve: 
            self.fork(reject,fn(lambda x: fn(x).fork(reject,resolve)))
        )
    def join(self):
        return self.chain(join)

In [293]:
encase(add)(1,2).fork(r.identity,r.add(80))
def join(x):
    return x.join()

result: 3


In [294]:
Task(encase(add)(1,'2').fork).fork(r.identity,r.identity)

result: unsupported operand type(s) for +: 'int' and 'str'


TypeError("unsupported operand type(s) for +: 'int' and 'str'")

In [295]:
Task.of(io_op.ap(of(10)).ap(of(-90))).fork(r.add(9),r.identity)

IO(unsafe_io=<function compose.<locals>.wrapper at 0x1122deb70>)

In [299]:
Task.of(io_op.ap(of(10)).ap(of(-90))).join()

Task(fork=<function Task.chain.<locals>.<lambda> at 0x1122de400>)

In [300]:
@dataclass
class Either:
    value: Any
    @classmethod
    def of(cls,value):
        return cls.right(value)

@dataclass
class Left(Either):
    def is_left(self):
        return True
    def is_right(self):
        return False
    @classmethod
    def of(self,value):
        raise NotImplemented
    def map(self,fn):
        return self
    def ap(self,fn):
        return self
    def chain(self,fn):
        return self 
    def join(self):
        return self
    def sequence(self,of):
        return of(self)
    def traverse(self,of,fn):
        return of(self)

@dataclass
class Right(Either):
    def is_left(self):
        return False
    def is_right(self):
        return True
    @classmethod
    def of(cls,value):
        raise NotImplementedError
    def map(self,fn):
        return Either.of(fn(self.value))
    def ap(self,f):
        return f.map(self.value)
    def chain(self,fn):
        return fn(self.value)
    def join(self):
        return self.value
    def sequence(self,of):
        return self.traverse(of,r.identity)
    def traverse(self,of,fn):
        return fn(self.value).map(Either.of)
    
Either.right = Right
Either.left = Left

In [386]:
Either.of(add).ap(Either.of(9))\
    .ap(Either.of(89)).map(sub).ap(Either.of(89))



adding!
subbing!


Right(value=9)

In [457]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import numpy as np
import time
import glob
import random
import string

def multithreading(func, args, workers):
    begin_time = time.time()
    with ThreadPoolExecutor(max_workers=workers) as executor:
        res = executor.map(func, args, [begin_time for i in range(len(args))])
    return list(res)

from urllib.request import urlopen

def download(url, base):
    start = time.time() - base
    try:
        resp = urlopen(url)
    except Exception as e:
        print ('ERROR: %s' % e)
    stop = time.time() - base
    return start,stop

# def curry(fn):
#     @ft.wraps(fn)
#     def wrapper(*arg,**kwargs):
#         try:
#             return fn(*arg,**kwargs)
#         except TypeError as e:
#             #print(e)
#             if hasattr(fn,'func'):
#                 return curry(ft.partial(fn.func,*fn.args,*arg,**fn.keywords,**kwargs))
#             else:
#                 return curry(ft.partial(fn,*arg,**kwargs))
#     return wrapper

# multithreading(download, urls, 2)

In [417]:
N = 6
URL = 'http://scholar.princeton.edu/sites/default/files/oversize_pdf_test_0.pdf'
urls = [URL for i in range(N)]

In [418]:
%timeit -n 1 [download(url, 1) for url in urls]


The slowest run took 5.58 times longer than the fastest. This could mean that an intermediate result is being cached.
1.88 s ± 1.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [420]:
dl_times = multithreading(download, urls, 1)

In [424]:
print([ (stop-start) for start,stop in dl_times ])
print(dl_times[-1][-1]-dl_times[0][0])

[0.2762460708618164, 0.4635169506072998, 0.4547419548034668, 0.4723970890045166, 0.09868907928466797, 0.07653093338012695]
1.853722095489502


In [425]:
dl_times_2 = multithreading(download, urls, 2)

In [426]:
print([ (stop-start) for start,stop in dl_times_2 ])
print(dl_times_2[-1][-1]-dl_times_2[0][0])

[0.6512303352355957, 0.4399139881134033, 0.24379897117614746, 0.1127309799194336, 0.17248821258544922, 0.18387794494628906]
0.9530880451202393


In [319]:
@dataclass
class Identity:
    value: Any
    @classmethod
    def of(cls,value):
        return cls(value)
    def map(self,fn):
        return Identity.of(fn(self.value))
    def ap(self,f):
        return f.map(self.value)
    def chain(self,fn):
        return self.map(fn).join()
    def join(self):
        return self.value
    def sequence(self,of):
        return self.traverse(of,r.identity)
    def traverse(self,of,fn):
        return fn(self.value).map(Identity.of)
    

In [433]:
out2 = ( 
Identity.of(lambda x,y: x+y)
    .map(dec) # decorate
    .map(curry) # decorate
    .ap(Identity.of(10)) # apply
    .ap(Identity.of(10)) # apply
)

args (10,),{}
args (10, 10),{}
20


In [407]:
from time import sleep

def narco(x):
    sleep(x)
    return f'done sleeping {x}... '
narco(1)+narco(2)

'done sleeping 1... done sleeping 2... '

In [413]:
Identity.of(op.add)\
    .map(dec) \
    .map(curry) \
    .ap(Identity.of(narco(5))) \
    .ap(Identity.of(narco(5)))

args ('done sleeping 5... ',),{}
args ('done sleeping 5... ', 'done sleeping 5... '),{}
done sleeping 5... done sleeping 5... 


Identity(value='done sleeping 5... done sleeping 5... ')

In [402]:
import asyncio
async def outer():
    await asyncio.sleep(3)
    print('done')
    async def inner():
        return 'yeah'
    return inner

loop = asyncio.get_event_loop()

In [403]:
#inner = await outer()

inner = loop.create_task(outer())

In [404]:
result = loop.create_task(inner())

TypeError: '_asyncio.Task' object is not callable

done


In [ ]:
print(result)

In [ ]:
@dataclass
class AsyncOp:
    value: Any
    @classmethod
    def of(cls,value):
        return cls(value)
    async def map(self,fn):
        return Identity.of(await fn(self.value))
    def ap(self,f):
        return f.map(self.value)
    def chain(self,fn):
        return self.map(fn).join()
    def join(self):
        return self.value
    def sequence(self,of):
        return self.traverse(of,r.identity)
    def traverse(self,of,fn):
        return fn(self.value).map(Identity.of)    

In [435]:
def make_compose(F,G):
    @dataclass
    class Compose:
        value: Any
        @classmethod
        def of(cls,x):
            return Compose(F(G(x)))
        def map(self,fn):
            return Compose(self.value.map(lambda x: x.map(fn)))
        def ap(self,f):
            return f.map(self.value)
    return Compose

In [443]:
IdCompose = make_compose(Identity.of,Identity.of)

In [446]:
of = IdCompose.of
IdCompose.of(add).ap(of(of(1)))

TypeError: 'Identity' object is not callable

In [673]:
def eencase(fn):
    class Encaser:
        def fork(reject,resolve):
            def wrapper(*args,**kwargs):
                try:
                    result = fn(*args,**kwargs)
                    return resolve(result)
                except TypeError as e:
                    m = re.match("\D*expected*\D*(\d)*\D*arguments\D*(\d)*", str(e))
                    result = e
                    if not m:
                        raise e
                    return reject(*args) if m else resolve(e)
            return wrapper
    return Encaser

def either(fn):
    @ft.wraps(fn)
    def wrapper(*args,**kwargs):
        return encase(fn)(*args,**kwargs).fork(Left,Right)
    return wrapper

def ecurry(fn):
    efn = Either.of(ft.partial(fn) ) if not isinstance(fn,Either) else fn
    #def wrapper(arg):
    return eencase(efn.ap).fork(lambda x,e: ecurry(ft.partial(op.add,x)),Right )
    #return wrapper

        

In [548]:
ecurry(op.add)(Either.of(1))

<function __main__.eencase.<locals>.Encaser.fork.<locals>.wrapper(*args, **kwargs)>

In [516]:
either(op.add)(1).ap(Either.of(2))

result: op_add expected 2 arguments, got 1


Left(value=TypeError('op_add expected 2 arguments, got 1',))

In [476]:
encase(curry(add))(1).fork(Left,Right)

result: <function add at 0x112401488>


Right(value=<function add at 0x112401488>)

In [594]:
#Either.of(encase(op.add)).ap(Either.of(1)).ap(Either.of(1))

In [595]:
eencase(op.add).fork(Left,Right)(1)

<class 'TypeError'> op_add expected 2 arguments, got 1


Left(value=1)

In [596]:
import inspect

In [600]:
# efn = Either.of(eencase(op.add).fork(lambda x,e: ft.partial(op.add,x),Right ))
# #.ap(Either.of(1))\
# #eencase(efn.ap).fork(lambda x,e: ft.partial(op.add,x),Right )(1)
# efn.ap(Either.of(1))

In [666]:
def throw(e):
    print('here')
    raise e

In [680]:
of = Either.of

ecurry2 = lambda fn: compose(eencase(fn.ap)\
    .fork( 
        compose(efn,of(partial).ap(fn).ap),  
        r.identity
    ), of)

#ecurry2(Either.of(op.add))(1)('1')

eencase(op.add).fork(r.identity,r.identity)(10)
#efn(Either.of(1))


10

In [565]:
@r.curry
def partial(x,y):
    return ft.partial(x,y)

In [632]:
Either.of(partial).ap(Either.of(op.add)).ap(Either.of(1))

Right(value=functools.partial(<built-in function add>, 1))

In [602]:
import re

In [ ]:
re.find('op_add expected 2 arguments, got 1')

In [630]:
t = 'op_add expected 2 arguments, got 1'
m = re.match("\D*expected*\D*(\d)*\D*arguments\D*(\d)*", t)
print(m)
if m:
    print(m.group(2))

<_sre.SRE_Match object; span=(0, 34), match='op_add expected 2 arguments, got 1'>
1


In [644]:
str(TypeError('a'))

'a'